In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [15]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px

today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=720)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

df = yf.download('GOOG', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
df["Date"] = df.index
df = df[["Date","Close"]]
df.reset_index(drop=True, inplace=True)
print(df.head())

                       Date       Close
0 2021-03-23 00:00:00-04:00  102.648003
1 2021-03-24 00:00:00-04:00  102.252998
2 2021-03-25 00:00:00-04:00  102.218002
3 2021-03-26 00:00:00-04:00  101.777496
4 2021-03-29 00:00:00-04:00  102.797501


preparing the dataset which can fit easily in the Linear Regression model:

In [16]:

def prepare_data(df,forecast_col,forecast_out,test_size):
    label = df[forecast_col].shift(-forecast_out) #creating new column called label with the last 5 rows are nan
    X = np.array(df[[forecast_col]]) #creating the feature array
    X = preprocessing.scale(X) #processing the feature array
    X_lately = X[-forecast_out:] #creating the column i want to use later in the predicting method
    X = X[:-forecast_out] # X that will contain the training and testing
    label.dropna(inplace=True) #dropping na values
    y = np.array(label)  # assigning Y
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_size, random_state=0) #cross validation

    response = [X_train,X_test , Y_train, Y_test , X_lately]
    return response

Now we need to prepare three input variables as already prepared in the function created in the above section. 
We need to declare an input variable mentioning about which column we want to predict. The next variable we need to 
declare is how much far we want to predict and the last variable that we need to declare is how much should be the 
size of the test set.

In [17]:
forecast_col = 'Close'
forecast_out = 5
test_size = 0.2

spliting the data and fit into the linear regression model:

In [18]:
X_train, X_test, Y_train, Y_test , X_lately =prepare_data(df,forecast_col,forecast_out,test_size); #calling the method were the cross validation and data preperation is in
learner = LinearRegression() #initializing linear regression model

learner.fit(X_train,Y_train) #training the linear regression model

LinearRegression()

predicting the output 

In [19]:
score=learner.score(X_test,Y_test)#testing the linear regression model
forecast= learner.predict(X_lately) #set that will contain the forecasted data
response={}#creting json object
response['test_score']=score
response['forecast_set']=forecast

print(response)

{'test_score': 0.9180414448442357, 'forecast_set': array([96.09375649, 94.7230622 , 95.18968359, 93.25516471, 91.65116343])}
